In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

<h4>Данные</h4>

In [65]:
country_names = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_COUNTRY.csv")

In [66]:
country_names

,COUNTRY_ID,COUNTRY_NAME_EN
0,AFG,Afghanistan
1,ALB,Albania
2,DZA,Algeria
3,ASM,American Samoa
4,AND,Andorra
...,...,...
236,WLF,Wallis and Futuna Islands
237,WSM,Samoa
238,YEM,Yemen
239,COD,Democratic Republic of the Congo


In [120]:
labels = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_students_label.csv")
# labels = labels['INDICATOR_ID'].astype(int)
labels.columns

Index(['INDICATOR_ID', 'INDICATOR_LABEL_EN'], dtype='object')

In [121]:
national = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_DATA_NATIONAL.csv")
national.columns

C:\Users\yuliy\AppData\Local\Temp\ipykernel_17988\3021277289.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  national = pd.read_csv(r"C:\Users\yuliy\Documents\UNESCO Education\OPRI_DATA_NATIONAL.csv")


Index(['indicator_id', 'country_id', 'year', 'value', 'magnitude',
       'qualifier'],
      dtype='object')

In [202]:
#данные въезда в США за 2022 г
usa_add_long = pd.read_csv(r"C:\Users\yuliy\Education_Projects\students_mobility\usa_inbound.csv")
usa_add_long['to'] = 'United States of America'
usa_add_long['year'] = '2022'
usa_add_long = usa_add_long.rename(columns={'unesco':'from'})
usa_add_long = usa_add_long[['from','to','year','2022']]
usa_add_long.columns = ['from','to','year','value']
usa_add_long = usa_add_long[usa_add_long['from']!='#Н/Д']
usa_add_long['value'] = usa_add_long['value'].fillna(0)
usa_add_long['value'] = usa_add_long['value'].astype(int)
usa_add_long['country_id'] = 'USA'
usa_add_long.head()

,from,to,year,value,country_id
0,Afghanistan,United States of America,2022,682,USA
1,Albania,United States of America,2022,1087,USA
2,Algeria,United States of America,2022,302,USA
3,Andorra,United States of America,2022,20,USA
4,Angola,United States of America,2022,553,USA


In [203]:
#данные въезда в Казахстан за 2022 г
kaz_add_long = pd.read_csv(r"C:\Users\yuliy\Education_Projects\students_mobility\kazakhstan_inbound.csv")
kaz_add_long['year'] = kaz_add_long['year'].astype(str)
kaz_add_long['country_id'] = 'KAZ'
kaz_add_long.head()

,from,to,year,value,country_id
0,Uzbekistan,Kazakhstan,2022,9571.0,KAZ
1,Tajikistan,Kazakhstan,2022,361.0,KAZ
2,Turkmenistan,Kazakhstan,2022,1373.0,KAZ
3,Azerbaijan,Kazakhstan,2022,116.0,KAZ
4,Kyrgyzstan,Kazakhstan,2022,597.0,KAZ


In [204]:
nld_add_long = pd.read_csv(r"C:\Users\yuliy\Education_Projects\students_mobility\nld_inbound.csv")
nld_add_long  = nld_add_long.dropna()
nld_add_long['year'] = nld_add_long['year'].astype(str)
nld_add_long['country_id'] = 'NLD'
nld_add_long.head()

,from,to,year,value,country_id
18,Belgium,Netherlands,2022,4780.0,NLD
29,Bulgaria,Netherlands,2022,5112.0,NLD
41,China,Netherlands,2022,5610.0,NLD
70,France,Netherlands,2022,4635.0,NLD
76,Germany,Netherlands,2022,22775.0,NLD


inbound отчеты - обработка и добавление отсутствующих стран

In [205]:
from_to_labels = labels[~labels['INDICATOR_ID'].isin([26637,26638,43188])]
from_to_labels['from'] = from_to_labels['INDICATOR_LABEL_EN'].apply(lambda x:x.split('from ')[1].split(', both sexes (number)')[0].replace('the ',''))
from_to_labels.head()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_17988\106753272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_to_labels['from'] = from_to_labels['INDICATOR_LABEL_EN'].apply(lambda x:x.split('from ')[1].split(', both sexes (number)')[0].replace('the ',''))


,INDICATOR_ID,INDICATOR_LABEL_EN,from
0,26421,"Africa: Students from Algeria, both sexes (num...",Algeria
1,26422,"Africa: Students from Angola, both sexes (number)",Angola
2,26423,"Africa: Students from Benin, both sexes (number)",Benin
3,26424,"Africa: Students from Botswana, both sexes (nu...",Botswana
4,26425,"Africa: Students from Burkina Faso, both sexes...",Burkina Faso


In [206]:
total_from_to = national[national['indicator_id'].isin(from_to_labels['INDICATOR_ID'].unique())]
total_from_to = pd.merge(total_from_to,from_to_labels,left_on='indicator_id',right_on='INDICATOR_ID',how='left')
total_from_to = pd.merge(total_from_to,country_names,left_on='country_id',right_on='COUNTRY_ID',how='left')
total_from_to = total_from_to.rename(columns={'COUNTRY_NAME_EN':'to'})
total_from_to =total_from_to[['indicator_id','country_id','year','value','from','to']]
total_from_to['value'] = total_from_to['value'].astype(int)
print(total_from_to.shape)
total_from_to.head()

(453280, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0,Algeria,Aruba
1,26421,ABW,2004,0,Algeria,Aruba
2,26421,ABW,2006,0,Algeria,Aruba
3,26421,ABW,2009,0,Algeria,Aruba
4,26421,ABW,2010,0,Algeria,Aruba


In [207]:
total_from_to[((total_from_to['to'] == 'United States of America') & (total_from_to['year'] == 2022))]

,indicator_id,country_id,year,value,from,to
175863,26503,USA,2022,0,United States,United States of America


In [208]:
total_from_to[((total_from_to['to'] == 'Netherlands') & (total_from_to['year'] == 2022))]

,indicator_id,country_id,year,value,from,to
372156,26599,NLD,2022,0,Netherlands,Netherlands


In [209]:
total_from_to[((total_from_to['to'] == 'Kazakhstan') & (total_from_to['year'] == 2022))]

,indicator_id,country_id,year,value,from,to


In [210]:
#удалим строки 2022  в США - пустые
total_from_to['year'] = total_from_to['year'].astype(str)
total_from_to = total_from_to[~((total_from_to['to'] == 'United States of America') & (total_from_to['year'] == '2022'))]
total_from_to = total_from_to[~((total_from_to['to']=='Netherlands') & (total_from_to['year']=='2022'))]
total_from_to = total_from_to[~((total_from_to['to']=='Kazakhstan') & (total_from_to['year']=='2022'))]
print(total_from_to.shape)
total_from_to.head()

(453278, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0,Algeria,Aruba
1,26421,ABW,2004,0,Algeria,Aruba
2,26421,ABW,2006,0,Algeria,Aruba
3,26421,ABW,2009,0,Algeria,Aruba
4,26421,ABW,2010,0,Algeria,Aruba


In [211]:
print(total_from_to.shape)
total_from_to = pd.concat([total_from_to,usa_add_long,kaz_add_long,nld_add_long])
print(total_from_to.shape)
total_from_to

(453278, 6)
(453549, 6)


,indicator_id,country_id,year,value,from,to
0,26421,ABW,2003,0.0,Algeria,Aruba
1,26421,ABW,2004,0.0,Algeria,Aruba
2,26421,ABW,2006,0.0,Algeria,Aruba
3,26421,ABW,2009,0.0,Algeria,Aruba
4,26421,ABW,2010,0.0,Algeria,Aruba
...,...,...,...,...,...,...
205,NaN,NLD,2022,2289.0,Turkey,Netherlands
210,NaN,NLD,2022,1095.0,Ukraine,Netherlands
212,NaN,NLD,2022,1846.0,United Kingdom of Great Britain and Northern I...,Netherlands
217,NaN,NLD,2022,1289.0,Viet Nam,Netherlands


<h4>Входящие по странам</h4>

На основании отчетов inbound в разрезе страна-страна

In [246]:
countries = total_from_to[total_from_to['year'].isin(['2017','2022'])]
countries = countries[countries['from']!=countries['to']]
countries = countries.drop_duplicates()
countries = countries.pivot(index=['from','to'], columns='year', values='value').reset_index()
countries = countries[countries['from']!='unknown countries']
countries

year,from,to,2017,2022
0,Afghanistan,Albania,NaN,0.0
1,Afghanistan,Andorra,0.0,NaN
2,Afghanistan,Argentina,15.0,16.0
3,Afghanistan,Armenia,0.0,0.0
4,Afghanistan,Australia,44.0,73.0
...,...,...,...,...
24441,Zimbabwe,United Kingdom of Great Britain and Northern I...,924.0,1548.0
24442,Zimbabwe,United Republic of Tanzania,NaN,16.0
24443,Zimbabwe,United States of America,1365.0,1789.0
24444,Zimbabwe,Uzbekistan,0.0,0.0


In [247]:
inbound = pd.merge(countries.groupby(['to'])['2017'].sum().reset_index(),
                 countries.groupby(['to'])['2022'].sum().reset_index(),
                 on = 'to',
                 how='left')
inbound.loc[inbound['to']=='Egypt','2017']=51162
inbound.loc[inbound['to']=='Thailand','2017'] = 43821
inbound.loc[inbound['to']=='Mexico','2022']=51659
inbound.loc[inbound['to']=='Iran (Islamic Republic of)','2022']=94406
# diff.loc[inbound['country_id']=='KAZ','2022']=26080
inbound.loc[inbound['to']=='Ukraine','2022']=50870
inbound.loc[inbound['to']=='Iran (Islamic Republic of)','to']='Iran'
inbound.loc[inbound['to']=='United Kingdom of Great Britain and Northern Ireland','to']='Great Britain'
inbound.loc[inbound['to']=='China, Hong Kong Special Administrative Region','to']='Hong Kong'
inbound.loc[inbound['to']=='China, Macao Special Administrative Region','to']='Macao'
inbound.loc[inbound['to']=='United States of America','to']='USA'
inbound.loc[inbound['to']=='United Republic of Tanzania','to']='Tanzania'
inbound.loc[inbound['to']=="Lao People's Democratic Republic",'to'] = 'Lao'
inbound.loc[inbound['to']=="Turks and Caicos Islands",'to'] = 'Turks and Caicos'
inbound = inbound[~(inbound['2017'].isna()) & ~(inbound['2022'].isna())]
inbound = inbound[~(inbound['2017']==0) & ~(inbound['2022']==0)]
inbound['2022/2017,%'] = round(inbound['2022']/inbound['2017']*100,0).astype(int)-100
inbound['2022/2017,%'] = inbound['2022/2017,%'].apply(lambda x:'+'+str(x) if x>0 else str(x))
inbound = inbound[inbound['2022']>5000]
def group(x):
    if x<20000:
        group_name = '5-20 тыс.'
    elif 20000<=x<50000:
        group_name = '20-50 тыс.'
    elif 50000<=x<100000:
        group_name = '50-100 тыс.'
    else:
        group_name = 'Больше 100 тыс.'
    return group_name
inbound['group'] = inbound['2022'].apply(lambda x: group(x))
inbound.set_index('to', inplace=True)
inbound.sort_values(by='2022/2017,%',ascending=False)
inbound

,2017,2022,"2022/2017,%",group
to,,,,
Argentina,85945.0,135754.0,+58,Больше 100 тыс.
Armenia,4138.0,6635.0,+60,5-20 тыс.
Australia,368594.0,364473.0,-1,Больше 100 тыс.
Austria,71967.0,82781.0,+15,50-100 тыс.
Azerbaijan,4264.0,6363.0,+49,5-20 тыс.
...,...,...,...,...
Ukraine,51872.0,50870.0,-2,50-100 тыс.
Great Britain,435035.0,674046.0,+55,Больше 100 тыс.
USA,965780.0,1035161.0,+7,Больше 100 тыс.


График входящих

In [244]:
n_cols = 2  # Number of columns in subplot grid
# unique_groups  = diff['group'].unique()
unique_groups = ['Больше 100 тыс.', '50-100 тыс.', '20-50 тыс.','5-20 тыс.']
num_plots = len(unique_groups)
num_cols = 2  # Maximum of 2 columns
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=unique_groups)

for i, group in enumerate(unique_groups):
    filtered_df = inbound[inbound['group'] == group]
    row_index = i // n_cols + 1
    col_index = i % n_cols + 1
    
    trace1 = go.Bar(x=filtered_df.index,
        y=filtered_df['2017'],
        name=f'2017 - Group {group}',
        text=[f"{value/1000:.1f}k" if value < 1000 else f"{int(value/1000)}k" for value in filtered_df['2017']],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=16),
        marker_color='blue',
        marker_line=dict(width=3, color=['red' if country == 'Russian Federation' else 'blue' for country in filtered_df.index]) 
        )
    trace2 = go.Bar(x=filtered_df.index,
        y=filtered_df['2022'],
        name=f'2022 - Group {group}',
        text=[f"{value/1000:.1f}k\n({filtered_df['2022/2017,%'].values[j]}%)" if value < 1000 else f"{int(value/1000)}k\n({filtered_df['2022/2017,%'].values[j]}%)" for j, value in enumerate(filtered_df['2022'])],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=16),
        marker_color='green',
        marker_line=dict(width=3, color=['red' if country == 'Russian Federation' else 'green' for country in filtered_df.index]) 
        )
    
    fig.add_trace(trace1, row=row_index, col=col_index)
    fig.add_trace(trace2, row=row_index, col=col_index)
        
fig.update_layout(
title='Входящий поток иностранных студентов по группам стран с 2017 по 2022 г.',
# xaxis_title='Country',
yaxis_title='Число студентов, тыс.',
barmode='group',  # Group bars together
height=1400,
width=2600,
plot_bgcolor='white'
)
for i in range(1, num_rows + 1):
    for j in range(1, num_cols + 1):
        fig.update_xaxes(tickangle=45, row=i, col=j)
        fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Входящий поток иностранных студентов по группам стран с 2017 по 2022 г.png")
fig.show()


На основании отчетов inbound в разрезе весь мир -страна

In [245]:
# total_inbound = national[national['indicator_id'].isin([26637])]
# total_inbound['from'] = 'World'
# total_inbound.head()

<h4>Исходящие по странам</h4>

OE.5T8.40510 
Total outbound internationally mobile tertiary students studying abroad, all countries, both sexes (UIS estimate) (number)

In [248]:
# total_outbound = national[national['indicator_id'].isin(['OE.5T8.40510'])]
# total_outbound = pd.merge(total_outbound,country_names,left_on='country_id',right_on='COUNTRY_ID',how='left')
# total_outbound.head()

In [304]:
world_outbound = total_outbound.groupby(['year'])['value'].sum().reset_index()
fig = px.bar(world_outbound,x='year',y='value',text='value')
fig.show()
# world_outbound

In [258]:
outbound = pd.merge(countries.groupby(['from'])['2017'].sum().reset_index(),
                 countries.groupby(['from'])['2022'].sum().reset_index(),
                 on = 'from',
                 how='left')
outbound.loc[outbound['from']=='Iran (Islamic Republic of)','from']='Iran'
outbound.loc[outbound['from']=='United Kingdom of Great Britain and Northern Ireland','from']='Great Britain'
outbound.loc[outbound['from']=='China, Hong Kong Special Administrative Region','from']='Hong Kong'
outbound.loc[outbound['from']=='China, Macao Special Administrative Region','from']='Macao'
outbound.loc[outbound['from']=='United States of America','from']='USA'
outbound.loc[outbound['from']=='United Republic of Tanzania','from']='Tanzania'
outbound.loc[outbound['from']=="Lao People's Democratic Republic",'from'] = 'Lao'
outbound.loc[outbound['from']=="Turks and Caicos Islands",'from'] = 'Turks and Caicos'
outbound = outbound[~(outbound['2017'].isna()) & ~(outbound['2022'].isna())]
outbound = outbound[~(outbound['2017']==0) & ~(outbound['2022']==0)]
outbound['2022/2017,%'] = round(outbound['2022']/outbound['2017']*100,0).astype(int)-100
outbound['2022/2017,%'] = outbound['2022/2017,%'].apply(lambda x:'+'+str(x) if x>0 else str(x))
outbound = outbound[outbound['2022']>5000]
def group(x):
    if x<20000:
        group_name = '5-20 тыс.'
    elif 20000<=x<50000:
        group_name = '20-50 тыс.'
    elif 50000<=x<100000:
        group_name = '50-100 тыс.'
    elif 100000<=x<200000:
        group_name = '100-200 тыс.'
    else:
        group_name = 'Больше 200 тыс.'
    return group_name
outbound['group'] = outbound['2022'].apply(lambda x: group(x))
outbound.set_index('from', inplace=True)
outbound.sort_values(by='2022/2017,%',ascending=False)
outbound

,2017,2022,"2022/2017,%",group
from,,,,
Afghanistan,28964.0,20145.0,-30,20-50 тыс.
Albania,17429.0,14542.0,-17,5-20 тыс.
Algeria,25235.0,34905.0,+38,20-50 тыс.
Angola,15252.0,9574.0,-37,5-20 тыс.
Argentina,8057.0,11679.0,+45,5-20 тыс.
...,...,...,...,...
Uruguay,4807.0,6017.0,+25,5-20 тыс.
Uzbekistan,34689.0,132963.0,+283,100-200 тыс.
Viet Nam,93589.0,133922.0,+43,100-200 тыс.


In [259]:
n_cols = 2  # Number of columns in subplot grid
# unique_groups  = diff['group'].unique()
unique_groups = ['Больше 200 тыс.','100-200 тыс.', '50-100 тыс.', '20-50 тыс.','5-20 тыс.']
num_plots = len(unique_groups)
num_cols = 2  # Maximum of 2 columns
num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate the number of rows
fig = make_subplots(rows=num_rows, cols=num_cols, subplot_titles=unique_groups)
outbound = outbound.sort_values(by=['2022'],ascending=False)
for i, group in enumerate(unique_groups):
    filtered_df = outbound[outbound['group'] == group]
    row_index = i // n_cols + 1
    col_index = i % n_cols + 1
    
    trace1 = go.Bar(x=filtered_df.index,
        y=filtered_df['2017'],
        name=f'2017 - Group {group}',
        text=[f"{value/1000:.1f}k" if value < 1000 else f"{int(value/1000)}k" for value in filtered_df['2017']],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=12),
        marker_color='brown',
        marker_line=dict(width=3, color=['black' if country == 'Russian Federation' else 'brown' for country in filtered_df.index]) 
        )
    trace2 = go.Bar(x=filtered_df.index,
        y=filtered_df['2022'],
        name=f'2022 - Group {group}',
        text=[f"{value/1000:.1f}k<br>({filtered_df['2022/2017,%'].values[j]}%)" if value < 1000 else f"{int(value/1000)}k<br>({filtered_df['2022/2017,%'].values[j]}%)" for j, value in enumerate(filtered_df['2022'])],  # Value labels in thousands
        textposition='auto',
        textfont=dict(size=12),
        marker_color='orange',
        marker_line=dict(width=3, color=['black' if country == 'Russian Federation' else 'orange' for country in filtered_df.index]) 
        )
    
    fig.add_trace(trace1, row=row_index, col=col_index)
    fig.add_trace(trace2, row=row_index, col=col_index)
        
fig.update_layout(
title='Исходящий поток иностранных студентов по группам стран с 2017 по 2022 г.',
# xaxis_title='Country',
yaxis_title='Число студентов, тыс.',
barmode='group',  # Group bars together
height=1600,
width=2600,
plot_bgcolor='white'
)
for i in range(1, num_rows + 1):
    for j in range(1, num_cols + 1):
        fig.update_xaxes(tickangle=45, row=i, col=j)
        fig.update_yaxes(gridcolor='lightgrey', row=i, col=j)
pio.write_image(fig, "Исходящий поток иностранных студентов по группам стран с 2017 по 2022 г.png")
fig.show()

<h4>Внутри ЦА и России</h4>

In [260]:
countries

year,from,to,2017,2022
0,Afghanistan,Albania,NaN,0.0
1,Afghanistan,Andorra,0.0,NaN
2,Afghanistan,Argentina,15.0,16.0
3,Afghanistan,Armenia,0.0,0.0
4,Afghanistan,Australia,44.0,73.0
...,...,...,...,...
24441,Zimbabwe,United Kingdom of Great Britain and Northern I...,924.0,1548.0
24442,Zimbabwe,United Republic of Tanzania,NaN,16.0
24443,Zimbabwe,United States of America,1365.0,1789.0
24444,Zimbabwe,Uzbekistan,0.0,0.0


In [261]:
unique_labels = list(set(countries['from']).union(set(countries['to'])))
label_to_index = {label: index for index, label in enumerate(unique_labels)}
countries['source'] = countries['from'].map(label_to_index)
countries['target'] = countries['to'].map(label_to_index)
countries

year,from,to,2017,2022,source,target
0,Afghanistan,Albania,NaN,0.0,195,70
1,Afghanistan,Andorra,0.0,NaN,195,185
2,Afghanistan,Argentina,15.0,16.0,195,219
3,Afghanistan,Armenia,0.0,0.0,195,220
4,Afghanistan,Australia,44.0,73.0,195,52
...,...,...,...,...,...,...
24441,Zimbabwe,United Kingdom of Great Britain and Northern I...,924.0,1548.0,66,160
24442,Zimbabwe,United Republic of Tanzania,NaN,16.0,66,15
24443,Zimbabwe,United States of America,1365.0,1789.0,66,18
24444,Zimbabwe,Uzbekistan,0.0,0.0,66,178


In [262]:
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_labels,  # Use defined labels here
        color="blue"           # Color for all nodes
    ),
    link=dict(
        source=countries['source'],   # Indices correspond to labels
        target=countries['target'],
        value=countries['2022']       # Use values from the 2022 column
    )
)])

fig.update_layout(
    title_text="Sankey Diagram of Country Flows", 
    font_size=10,
    height=1200,
    width=1200,
    plot_bgcolor='white'
)
fig.show()